# C# Features

| C#    | SDK    |
| :---- | :----- |
| C# 12 | .NET 8 |
| C# 11 | .NET 7 |
| C# 10 | .NET 6 |

In [3]:
Console.WriteLine($"Running .NET CLR {System.Environment.Version}");

You running .NET CLR 8.0.0


## C# 12
- primary constructors
- collection expressions
- ref readonly parameters
- default lambda parameters
- alias any type
- inline arrays
- experimental attribute
- interceptors (experimental)

https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-12
https://learn.microsoft.com/en-us/dotnet/core/whats-new/dotnet-8

In [16]:
/*
    C# 12 features

    Primary constructors:
    - parameters are auto private fields
    - Employee's id not publicly accessible
    - need to becareful reusing parameter names
      - both classes have their own copies of id

    convenient to avoid declaring parameters
    might be problematic if there are multiple
    levels of inheritance with primary constructors
    might make it difficult to trace parameters
*/

public class Employee(int id)
{
    public decimal Id => id;

    public override string ToString() => $"Id: {id}";
}

public class Manager(int id, string rank) : Employee(id)
{
    public string Rank => rank;

    public override string ToString() => $"Id: {id}, Rank: {rank}";

    public void ChangeId(int newId) => id = newId;
}

public class Director : Employee
{
    public Director(int id) : base(id)
    {
        // Director will not have auto private field of id
        // Director class will reuse Employee's Id
    }
}

Employee emp = new(1);
// emp.Id = 5; // read only; no assignment
Console.WriteLine($"Employee Id: {emp.Id}");

Manager mgr = new(5, "assistant");
Console.WriteLine($"Manager: {mgr}");

// both classes separate copies of id
// updating one doesn't change the other
mgr.ChangeId(20);
Console.WriteLine($"Manager: {mgr}");
Employee emp2 = mgr as Employee;
Console.WriteLine($"Employee: {emp2.Id}"); // still show as 5



Employee Id: 1
Manager: Id: 5, Rank: assistant
Manager: Id: 20, Rank: assistant
Employee: 5


In [20]:
/*
    C# 12 features
    
    Collection expressions
    - spread operator ..
*/

int[] array = [ 1, 2, 3];
List<int> list = [1, 2, 3];
int[] combined = [..array, ..list];
Console.WriteLine($"Length of combined: {combined.Length}");

// int[][] combined2d = [array, list]; // compiler complains list not correct type
int[][] combined2d = [array, array];
Console.WriteLine($"Combined length: {combined2d.Length}");
Console.WriteLine($"Combined depth: {combined2d[0].Length}");

// this hints that this syntax work by enumerating all values then combining into array 
// todo: check on mem allocations
int[] combined3 = [..array, ..System.Linq.Enumerable.Range(0, 10).Where(x => x % 2 == 0)];
Console.WriteLine($"Combined3: {string.Join(' ', combined3)}")



Length of combined: 6
Combined length: 2
Combined depth: 3
Combined3: 1 2 3 0 2 4 6 8
